In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Carregar o dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'

# Nome das colunas
column_names_pt_br = [
    'classe', 'alcool', 'acido_malico', 'cinzas', 'alcalinidade_de_cinzas',
    'magnesio', 'fenois_totais', 'flavanoides', 'fenois_nao_flavanoides',
    'proantocianinas', 'intensidade_de_cor', 'matiz',
    'od280_od315_de_vinhos_diluidos', 'prolina'
]

# Ler os dados
df = pd.read_csv(url, names=column_names_pt_br, dtype={'classe': object})

# Separar X e y
X = df.drop(columns=['classe'])
y = df['classe']

# Função para realizar o experimento de K-Fold Cross-Validation
def executar_kfold_experimento(random_state, k_values, n_splits=10):
    resultados = {}
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    for k in k_values:
        acc_scores = []

        for train_index, test_index in kf.split(X):
            # Dividir os dados em treino e teste
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            # Criar e treinar o modelo
            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, y_train)

            # Prever e calcular a acurácia
            y_pred = knn.predict(X_test)
            acc_scores.append(accuracy_score(y_test, y_pred))

        # Calcular média e desvio padrão das acurácias
        resultados[k] = {
            'media': np.mean(acc_scores),
            'desvio_padrao': np.std(acc_scores)
        }

    return resultados

# Configurações do experimento
random_states = [42, 17, 24]
k_values = [3, 5]

# Executar os experimentos para cada random_state
todos_resultados = {}

for random_state in random_states:
    print(f"Resultados para random_state={random_state}")
    resultados = executar_kfold_experimento(random_state, k_values)
    todos_resultados[random_state] = resultados

    for k, metrics in resultados.items():
        print(f"  k={k}: Média={metrics['media']:.4f}, Desvio Padrão={metrics['desvio_padrao']:.4f}")

# Análise Final
melhor_configuracao = []

for random_state, resultados in todos_resultados.items():
    melhor_k = max(resultados, key=lambda k: resultados[k]['media'])
    melhor_configuracao.append((random_state, melhor_k, resultados[melhor_k]['media']))

print("\nAnálise Final:")
for random_state, melhor_k, media in melhor_configuracao:
    print(f"Random State: {random_state} | Melhor k: {melhor_k} | Acurácia Média: {media:.4f}")

Resultados para random_state=42
  k=3: Média=0.7036, Desvio Padrão=0.1392
  k=5: Média=0.6641, Desvio Padrão=0.1001
Resultados para random_state=17
  k=3: Média=0.6987, Desvio Padrão=0.1040
  k=5: Média=0.6876, Desvio Padrão=0.1121
Resultados para random_state=24
  k=3: Média=0.7127, Desvio Padrão=0.0968
  k=5: Média=0.7127, Desvio Padrão=0.0940

Análise Final:
Random State: 42 | Melhor k: 3 | Acurácia Média: 0.7036
Random State: 17 | Melhor k: 3 | Acurácia Média: 0.6987
Random State: 24 | Melhor k: 3 | Acurácia Média: 0.7127
